
## 2.2 数据模块：Dataset和Dataloader

Dataset和DataLoader都是用来帮助我们加载数据集的两个重要工具类。Dataset用来构造支持索引的数据集。在训练时需要在全部样本中拿出小批量数据参与每次的训练，因此我们需要使用DataLoader，即DataLoader是用来在Dataset里取出一组数据(mini-batch)供训练时快速使用的。
本章将分为Dataset和DataLoader两节来介绍基本使用方法

### 2.2.1 Dataset简介及用法
上面我们提到，Dataset作为一个抽象类，需要定义其子类来实例化。所以我们需要自己定义其子类或者使用已经定义好的子类

1.自己定义其子类，其中pass为占位符

定义一个MyDataset类继承Dataset抽象类，并且改写其中的三个方法

In [1]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    
    def __init__(self):
        pass
    
    def __getitem__(self, index):
        pass
    
    def __len__(self):
        pass

在创建的dataset类中可根据自己的需求对数据进行处理，以时间序列使用为示例，输入3个时间步，输出1个时间步，batch_size=5

In [2]:
import torch 
from torch.utils.data import Dataset

class GetTrainTestData(Dataset):
    def __init__(self, input_len, output_len, train_rate, is_train=True):
        super().__init__()
        # 使用sin函数返回10000个时间序列,如果不自己构造数据，就使用numpy,pandas等读取自己的数据为x即可。
        # 以下数据组织这块既可以放在init方法里，也可以放在getitem方法里
        self.x = torch.sin(torch.arange(0, 1000, 0.1))
        self.sample_num = len(self.x)
        self.input_len = input_len
        self.output_len = output_len
        self.train_rate = train_rate
        self.src, self.trg = [], []
        if is_train:
            for i in range(int(self.sample_num*train_rate)-self.input_len-self.output_len):
                self.src.append(self.x[i:(i+input_len)])
                self.trg.append(self.x[(i+input_len):(i+input_len+output_len)])
        else:
            for i in range(int(self.sample_num*train_rate), self.sample_num-self.input_len-self.output_len):
                self.src.append(self.x[i:(i+input_len)])
                self.trg.append(self.x[(i+input_len):(i+input_len+output_len)])
        print(len(self.src), len(self.trg))

    def __getitem__(self, index):
        return self.src[index], self.trg[index]

    def __len__(self):
        return len(self.src)  # 或者return len(self.trg), src和trg长度一样

实例化我们定义好的Dataset子类GetTrainTestData

In [3]:
data_train = GetTrainTestData(input_len=3, output_len=1, train_rate=0.8, is_train=True)
data_test = GetTrainTestData(input_len=3, output_len=1, train_rate=0.8, is_train=False)


7988 7988
1994 1994


2.已经定义好的内置子类

这个类的实例化有两个参数，分别为data_tensor (Tensor) 样本数据
和target_tensor (Tensor) 样本标签。下面我们举一个例子来使用TensorDataset

In [4]:
import torch
from torch.utils.data import TensorDataset

src = torch.sin(torch.arange(1, 1000, 0.1))
trg = torch.cos(torch.arange(1, 1000, 0.1))

于是可以直接实例化已定义好的Dataset子类TensorDataset

In [5]:
data = TensorDataset(src, trg)

### 2.2.2 Dataloader简介及用法

Dataset和DataLoader是一起使用的，Dataset来加载数据，DataLoader来迭代加载进来的数据，DataLoader本质上就是用来将已经加载好的数据以模型能够接收的方式输入到即将训练的模型中去

将上节中使用Dataset读取的数据输入到DataLoader中作为示例

In [6]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class GetTrainTestData(Dataset):
    def __init__(self, input_len, output_len, train_rate, is_train=True):
        super().__init__()
        # 使用sin函数返回10000个时间序列,如果不自己构造数据，就使用numpy,pandas等读取自己的数据为x即可。
        # 以下数据组织这块既可以放在init方法里，也可以放在getitem方法里
        self.x = torch.sin(torch.arange(1, 1000, 0.1))
        self.sample_num = len(self.x)
        self.input_len = input_len
        self.output_len = output_len
        self.train_rate = train_rate
        self.src,  self.trg = [], []
        if is_train:
            for i in range(int(self.sample_num*train_rate)-self.input_len-self.output_len):
                self.src.append(self.x[i:(i+input_len)])
                self.trg.append(self.x[(i+input_len):(i+input_len+output_len)])
        else:
            for i in range(int(self.sample_num*train_rate), self.sample_num-self.input_len-self.output_len):
                self.src.append(self.x[i:(i+input_len)])
                self.trg.append(self.x[(i+input_len):(i+input_len+output_len)])
        print(len(self.src), len(self.trg))

    def __getitem__(self, index):
        return self.src[index], self.trg[index]

    def __len__(self):
        return len(self.src)  # 或者return len(self.trg), src和trg长度一样


data_train = GetTrainTestData(input_len=3, output_len=1, train_rate=0.8, is_train=True)
data_test = GetTrainTestData(input_len=3, output_len=1, train_rate=0.8, is_train=False)
data_loader_train = DataLoader(data_train, batch_size=5, shuffle=False)
data_loader_test = DataLoader(data_test, batch_size=5, shuffle=False)


7988 7988
1994 1994


In [9]:
for idx, train in enumerate(data_loader_train):
    print(idx, train)
    break

0 [tensor([[0.8415, 0.8912, 0.9320],
        [0.8912, 0.9320, 0.9636],
        [0.9320, 0.9636, 0.9854],
        [0.9636, 0.9854, 0.9975],
        [0.9854, 0.9975, 0.9996]]), tensor([[0.9636],
        [0.9854],
        [0.9975],
        [0.9996],
        [0.9917]])]
